# Day 17: Trick Shot
* Peak height seems like an easy calculation as the upward velocity just drops by 1 each step and once we hit zero it wont go higher.
* The hit tests is a little more involved but seems also pretty straight forward
    * Basically we know that X will trend toward 0 and then settle there from either direction, so we can easily calculate if X will ever make it to our target and if it does, how many steps it'll take. For now we'll assume our X velocity will always be positive.
    * For Y, our gravity constant is 1 which makes this easier. In other words we don't even need to check steps per say as Y will always hit 0 again before going negative. I.E, if our initial velocity is 2, our velocity over time will be 2 -> 1 -> 0 -> -1 -> -2 -> -3 -> -4 etc. This means our points after origin will be 2 -> 3 -> 3 -> 2 -> 0 -> -3 -> -7 etc. In this case it'd take 5 steps to be back to y=0 and then start dropping.
        * If initial velocity is 3 then we get 3 -> 2 -> 1 -> 0 -> -1 -> -2 -> -3 -> -4, and 3 -> 5 -> 6 -> 6 -> 5 -> 3 -> 0 -> -4. That's 7 steps til touching origin again and our velocity from there is essentially -(1+v). 
        * Thought it may not matter, we can calculate the steps lost in upward velocity before we hit origin again by doing 2v+1
        * So this is an easier calculation as we know our upward cancels in proportion to our start and from then provides us with an initial downward velocity to begin calculating for target breach. Which then simply becomes the same exercise as what we do for X

In [ ]:
let getPeak ((x1:int,x2:int),(y1,y2)) (vx:int, vy:int) =
    //  Check if our X velocity will ever be enough to get us to target
    let xSteps = Seq.unfold (fun (x,v) -> if v = 0 then None else match x + v with i when i > x2 -> None | i -> Some (i, (i, v - 1))) (0,vx)
    if Seq.isEmpty xSteps then None
    elif (Seq.last xSteps) < x1 then None
    //  X trends toward 0 and stops so if we don't fall short we'll likely hit it
    //  There is the posibility that our initial X velocity will overshoot the target on step 1, but we'll avoid that by not checking velocities
    //      that could put us in that situation and thus simplifying our calculation
    else
        // On to calculating Y now
        let startingDownwardVelocity = -(1+vy)
        // So let's calculate all the points between our downward vel and our target to see if we get there
        let ySteps = Seq.unfold (fun (y,v) -> match y + v with i when i < y1 -> None | i -> Some (i, (i, v - 1))) (0, startingDownwardVelocity)
        if Seq.isEmpty ySteps then None
        elif (Seq.last ySteps) > y2 then None
        // If we got this far, then we'll land within our target so let's calculate our peak
        else 
            Seq.unfold (fun (y,v) -> if v = 0 then None else Some (y + v, (y + v,v - 1))) (0, vy)
            |> Seq.last |> Some
        
let findCandidates (x1:int,x2:int) (y1:int,y2:int) =
    //  We know that any X velocity beyond our furthest target wont do any good so skip that
    //  Same goes for our Y velocity but that is translated so we'll skip any -(v+1) velocities beyond our lowest point

    [|
        for x = 0 to x2 do
            for y = 0 downto y1 + 1 do
                getPeak ((x1,x2),(y1,y1)) (x,y)
    |]

findCandidates (20,30) (-10,-5)

Error: System.ArgumentException: The input sequence was empty. (Parameter 'source')
   at Microsoft.FSharp.Collections.SeqModule.Last[T](IEnumerable`1 source)
   at FSI_0066.getPeak(Tuple`2 _arg1, Tuple`2 _arg2, Int32 vx, Int32 vy)
   at FSI_0066.findCandidates(Int32 x1, Int32 x2, Int32 y1, Int32 y2)
   at <StartupCode$FSI_0066>.$FSI_0066.main@()

In [ ]:
open System.Text.RegularExpressions

type Bounds (coords:List<int>) =
    let [x1;x2;y1;y2] = coords
    member this.UL = (x1,y2)
    member this.UR = (x2,y2)
    member this.LL = (x1,y1)
    member this.LR = (x2,y2)
    member this.HitTest (x:int,y:int) :bool = x >= x1 && x <= x2 && y >= y1 && y <= y2
    member this.IsBeyond (x:int,y:int) :bool = x > x2 || y < y1

let (|Reg|_|) pattern input =
   let m = Regex.Match(input,pattern)
   if (m.Success) then Some(List.tail [ for g in m.Groups -> g.Value] |> List.map int) else None

let parse str =
    match str with
    | Reg @"target area: x=(\d+)\.\.(\d+), y=(-\d+)..(-\d+)" a -> a
    | _ -> failwith "Bad input"

let inline step (px,py) (vx,vy) = (px+vx, py+vy)
let inline adjustVel (vx,vy) = ((if vx > 0 then vx - 1 elif vx < 0 then vx + 1 else 0), vy - 1)

let sampleTargetArea = "target area: x=20..30, y=-10..-5"

let target = parse sampleTargetArea |> Bounds

let rec go point vel s =
    if s > 100 then None
    let newPoint = step point vel
    match target.IsBeyond newPoint with
    | true -> None
    | false ->
        match target.HitTest newPoint with
        | true -> Some s
        | false ->
            let newVel = adjustVel vel
            go newPoint newVel (s + 1)

go (0,0) (7,2) 0

Stopped due to error


Error: input.fsx (4,9)-(4,22) typecheck warning Incomplete pattern matches on this expression. For example, the value '[_;_;_;_;_]' may indicate a case not covered by the pattern(s).
input.fsx (29,21)-(29,25) typecheck error This 'if' expression is missing an 'else' branch. Because 'if' is an expression, and not a statement, add an 'else' branch which also returns a value of type ''a option'.

In [ ]:
Array.append [|10..-1..0|] [|-10..0|]

index,value
0,10
1,9
2,8
3,7
4,6
5,5
6,4
7,3
8,2
9,1
